### Based on the paper from Goodwill et al.

In [60]:
import numpy as np
from tqdm import tqdm
import seaborn as sns
import pandas as pd
from tools.nbloader import NotebookLoader
from scipy import stats

trace_len = 50
sample_size = 350

def gen_trace(key=0, mu=128):
    return np.array([np.random.normal(mu + (key/256 - 1), 1, trace_len) for _ in range(sample_size)], dtype=np.uint8)

def gen_mask_trace(mu=128):
    random_keys = np.random.uniform(0, 256, sample_size) / 256

    return np.array([np.random.normal(mu + (rk - 1), 1, trace_len) for rk in random_keys], dtype=np.uint8)

tg_a1 = gen_trace(0)
tg_a2 = gen_trace(256)
tg_b = gen_mask_trace()

gen_mask_trace()

array([[127, 127, 126, ..., 125, 127, 129],
       [126, 126, 127, ..., 128, 125, 128],
       [127, 126, 127, ..., 125, 126, 126],
       ...,
       [128, 125, 126, ..., 127, 127, 128],
       [126, 125, 128, ..., 126, 125, 128],
       [126, 127, 127, ..., 126, 127, 125]], dtype=uint8)

In [61]:
def confidence_level(all_t_values, p=.95):
    tvs_flat = abs(all_t_values.flatten())
    n = round(len(tvs_flat) * p)

    return sorted(tvs_flat)[n]

def device_fails(ta: np.array, tb: np.array, c=2.5):
    """
    @param ta should be a positive-valued array of t-test results from some power trace with given key.
    @param tb should be a positive-valued array of t-test results from a different set of traces with the same key.
    @param c should be the confidence value.
    """
    return np.array((ta >= c) * (tb >= c)).any()

def t_test(a, b):
    num = a.mean(axis=0) - b.mean(axis=0)
    den = (((a.var(axis=0) / len(a)) + (b.var(axis=0) / len(b))) ** .5)

    return np.array(num / den)

def stats_t_test(a, b):
    return stats.ttest_ind(a, b, equal_var=False)[0]

def gen_rvs(mean=5):
    return lambda: np.array([stats.norm.rvs(loc=mean,scale=.1,size=50) for _ in range(50)])

def tvla(test, lefts, rights):
    a, b, c, d = lefts
    x, y = rights

    tk1 = abs(np.array(test(a, c)))
    tk2 = abs(np.array(test(b, d)))
    tm1 = abs(np.array(test(a, x)))
    tm2 = abs(np.array(test(b, y)))

    confidence = np.percentile(np.array([*tk1, *tk2, *tm1, *tm2]).flatten(), 95)

    return device_fails(tk1, tk2, confidence), device_fails(tm1, tm2, confidence)

def bench(test, gen_one, gen_two, total=100):
    acc = np.array([(False, False)] * total)
    for ix in tqdm(range(total)):
        acc[ix] = tvla(test, [gen_one() for _ in range(4)], (gen_two(), gen_two()))

    return np.array(acc).sum(axis=0)

In [62]:
bench(t_test, gen_rvs(5), gen_rvs(6))

100%|██████████| 100/100 [00:01<00:00, 84.76it/s]


array([ 0, 36])

In [63]:
bench(t_test, gen_trace, gen_mask_trace)

100%|██████████| 100/100 [00:01<00:00, 88.30it/s]


array([ 0, 44])

In [66]:
trc = np.array([round(np.random.uniform(0, 255)) for _ in range(100000)])

def ctable_mv(ctable: np.array, num_observations: int):
    """
    Calculates the mean and variance from a contingency table with a corresponding number of observations.
    """
    # Using 128-bit floats prevents some rounding errors when comparing with the np implementation of var and mean.
    ixs = np.arange(0, len(ctable), dtype=np.float128)

    mu = (ctable * ixs).sum() / num_observations
    sigma2 = ((ctable * ixs ** 2).sum() / num_observations) - (mu ** 2)

    return mu, sigma2

print(ctable_mv(np.bincount(trc, minlength=256), len(trc)))

# Actual mean and variance
trc.mean(), trc.var()

(127.67098, 5409.7362458396000005)


(127.67098, 5409.736245839599)

In [ ]:
# Bench for CTable



In [ ]:
def plot_longform(traces):
    sns.set_style('whitegrid')

    longform = []
    for trace in traces:
        for ix in range(len(trace)):
            longform.append((ix, trace[ix]))

    cols = ["Sample point", "Power"]
    df = pd.DataFrame(longform, columns=cols)
    sns.lineplot(data=df, x=cols[0], y=cols[1])

In [ ]:
plot_longform(gen_trace(0))
plot_longform(gen_mask_trace())
plot_longform(gen_trace(256))